#PARTE 1: CONCEPTOS BÁSICOS DE BÚSQUEDAS ELÁSTICAS

*. ¿Qué es Elasticsearch/openSearch?
----------------------
ElasticSearch y OpenSearch son motores de búsqueda distribuidos basados en Apache Lucene, diseñados para manejar grandes volúmenes de texto de forma rápida mediante índices y documentos JSON.

1.   Características principales:
- Búsquedas en tiempo real
- Escalabilidad horizontal
- Búsquedas full-text
- Análisis de datos (agregaciones)
- RESTful API

2. Conceptos clave:
----------------
- ÍNDICE (Index): Colección de documentos similares (como una base de datos)
- DOCUMENTO (Document): Unidad básica de información en formato JSON
- CAMPO (Field): Par clave-valor dentro de un documento
- MAPPING: Define el esquema y tipos de datos de los campos
- SHARDS: División del índice para distribuir datos
- REPLICAS: Copias de los shards para alta disponibilidad

3.  Que tipo de búsquedas se pueden realizar:
-------------------
- TERM: Búsqueda exacta (no analizada)
- MATCH: Búsqueda full-text (analizada)
- BOOL: Combina múltiples consultas (must, should, must_not, filter)
- RANGE: Búsqueda por rangos
- WILDCARD: Búsqueda con comodines
- FUZZY: Búsqueda con tolerancia a errores

4. Agregaciones (Aggregations):
-----------------------------
- METRICS: Cálculos numéricos (sum, avg, min, max)
- BUCKET: Agrupaciones de documentos
- PIPELINE: Agregaciones sobre otras agregaciones

# PARTE 2: INSTALACIÓN DE ELASTICSEARCH EN GOOGLE COLAB

In [2]:
# Instalar Elasticsearch y cliente Python
!pip install elasticsearch==8.11.0 -q

import os
import json
import requests
from datetime import datetime
from elasticsearch import Elasticsearch

In [3]:
# habilitamos drive de google desde colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Definir ruta de instalación
ES_PATH = "/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/ElasticSearch"

# Crear directorio si no existe
os.makedirs(ES_PATH, exist_ok=True)

print(" Directorio creado/verificado:", ES_PATH)

✅ Directorio creado/verificado: /content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/Ejercicios_de_clase/DataBase/ElasticSearch


In [5]:
# 3. DESCARGAR E INSTALAR ELASTICSEARCH
# ============================================================================

# Cambiar al directorio de trabajo
os.chdir(ES_PATH)

# Descargar Elasticsearch (versión 8.11.0)
ES_VERSION = "8.11.0"
ES_DOWNLOAD = f"elasticsearch-{ES_VERSION}-linux-x86_64.tar.gz"
ES_URL = f"https://artifacts.elastic.co/downloads/elasticsearch/{ES_DOWNLOAD}"

if not os.path.exists(f"elasticsearch-{ES_VERSION}"):
    print(f" Descargando Elasticsearch {ES_VERSION}...")
    !wget -q {ES_URL}
    print(" Extrayendo archivos...")
    !tar -xzf {ES_DOWNLOAD}
    !rm {ES_DOWNLOAD}
    print(" Elasticsearch instalado correctamente")
else:
    print(" Elasticsearch ya está instalado")

✅ Elasticsearch ya está instalado


In [6]:
# 4. CONFIGURAR ELASTICSEARCH
# ============================================================================

ES_HOME = f"{ES_PATH}/elasticsearch-{ES_VERSION}"
CONFIG_FILE = f"{ES_HOME}/config/elasticsearch.yml"

# Configuración básica para Colab
config = """
cluster.name: colab-cluster
node.name: colab-node-1
path.data: {}/data
path.logs: {}/logs
network.host: 127.0.0.1
http.port: 9200
xpack.security.enabled: false
xpack.security.enrollment.enabled: false
""".format(ES_HOME, ES_HOME)

with open(CONFIG_FILE, 'w') as f:
    f.write(config)

print("✅ Configuración de Elasticsearch actualizada")

✅ Configuración de Elasticsearch actualizada


In [ ]:
# 5. INICIAR ELASTICSEARCH
# ============================================================================

import subprocess
import time

# Iniciar Elasticsearch en background
print("🚀 Iniciando Elasticsearch...")
es_process = subprocess.Popen(
    [f"{ES_HOME}/bin/elasticsearch"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Esperar a que inicie
print("⏳ Esperando a que Elasticsearch esté listo...")
for i in range(30):
    try:
        response = requests.get("http://127.0.0.1:9200")
        if response.status_code == 200:
            print("✅ Elasticsearch está listo!")
            break
    except:
        time.sleep(2)
        print(f"   Intento {i+1}/30...")

# Conectar con Elasticsearch
es = Elasticsearch(["http://127.0.0.1:9200"])
print("\n📊 Info del cluster:", es.info())

# PARTE 3: CONEXIÓN A ELASTICSEARCH CLOUD

In [8]:
# Configuración de conexión
CLOUD_URL = "https://74b8ecb51d3747d09ed984342aa62aed.us-central1.gcp.cloud.es.io:443"
API_KEY = "OHN6cGRKb0IyWjRDUlg1NDJidWY6SVIzaFk0ZGNvcWYtOE52dm5sRUY1dw=="

# Crear cliente de ElasticSearch
es = Elasticsearch(
    CLOUD_URL,
    api_key=API_KEY,
    verify_certs=True
)

# Verificar conexión
if es.ping():
    print("Conexión exitosa a ElasticSearch Cloud")
    print(f"Cluster info: {es.info()['version']['number']}")
else:
    print("Error al conectar con ElasticSearch")


✓ Conexión exitosa a ElasticSearch Cloud
Cluster info: 9.2.1


## 3.1: PREPARACIÓN DEL INDEX Y CARGA DE DATOS

In [9]:
INDEX_NAME = "index_cuentos"

# Eliminar index si existe (para empezar limpio)
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' eliminado")

Index 'index_cuentos' eliminado


## 3.2_Crear mapping para el index


In [11]:
mapping = {
    "mappings": {
        "properties": {
            "texto": {"type": "text", "analyzer": "spanish"},
            "autor": {"type": "keyword"},
            "fecha_creacion": {"type": "date"},
            "tipo_cuento": {"type": "keyword"}
        }
    }
}
es.indices.create(index=INDEX_NAME, body=mapping)
print(f" Index '{INDEX_NAME}' creado con mapping")

✓ Index 'index_cuentos' creado con mapping


## 3.3_indexar (cargar) documentos al index

In [12]:
# Dataset de cuentos para cargar
cuentos = [
    {
        "texto": "Había una vez un pequeño conejo llamado Tambor que vivía en el bosque encantado. Un día decidió explorar más allá de su madriguera y encontró un jardín mágico lleno de zanahorias gigantes.",
        "autor": "María González",
        "fecha_creacion": "2024-01-15",
        "tipo_cuento": "infantil"
    },
    {
        "texto": "La casa en la colina estaba abandonada desde hace décadas. Los vecinos decían que por las noches se escuchaban gritos escalofriantes. Una noche, Juan decidió investigar y lo que encontró dentro le heló la sangre.",
        "autor": "Carlos Ruiz",
        "fecha_creacion": "2024-02-20",
        "tipo_cuento": "terror"
    },
    {
        "texto": "En el reino de Eldoria, la princesa Luna poseía el poder de controlar las estrellas. Cuando el dragón oscuro amenazó el reino, ella reunió a los guerreros elementales para salvar su mundo.",
        "autor": "Ana Martínez",
        "fecha_creacion": "2024-03-10",
        "tipo_cuento": "fantástico"
    },
    {
        "texto": "El alcalde prometió durante su campaña combatir la corrupción. Sin embargo, documentos filtrados revelaron su conexión con empresas fantasma. La población salió a las calles exigiendo transparencia.",
        "autor": "Roberto Silva",
        "fecha_creacion": "2024-04-05",
        "tipo_cuento": "político"
    },
    {
        "texto": "Pepito el elefante no podía volar como los pájaros, pero tenía el corazón más grande del bosque. Cuando sus amigos estuvieron en peligro, descubrió que su verdadero poder estaba en su valentía y amistad.",
        "autor": "Laura Ramírez",
        "fecha_creacion": "2024-05-12",
        "tipo_cuento": "infantil"
    },
    {
        "texto": "El espejo antiguo apareció en el mercado de pulgas. Quien se miraba en él no veía su reflejo, sino su peor pesadilla. La última persona que lo compró nunca volvió a ser la misma.",
        "autor": "Carlos Ruiz",
        "fecha_creacion": "2024-06-18",
        "tipo_cuento": "terror"
    },
    {
        "texto": "El mago Aldrin guardaba en su torre el libro de los hechizos perdidos. Cuando el portal dimensional se abrió, criaturas de otros mundos invadieron la tierra y solo él podía cerrar la brecha.",
        "autor": "Ana Martínez",
        "fecha_creacion": "2024-07-22",
        "tipo_cuento": "fantástico"
    },
    {
        "texto": "La reforma tributaria dividió al congreso. Mientras unos defendían beneficios para grandes empresas, otros exigían mayor equidad. El debate reveló las profundas fracturas del sistema político.",
        "autor": "Roberto Silva",
        "fecha_creacion": "2024-08-30",
        "tipo_cuento": "político"
    }
]


In [13]:
# Cargar documentos en el index
for i, cuento in enumerate(cuentos):
    response = es.index(index=INDEX_NAME, id=i+1, document=cuento)
    print(f"✓ Cuento {i+1} indexado: {response['result']}")

✓ Cuento 1 indexado: created
✓ Cuento 2 indexado: created
✓ Cuento 3 indexado: created
✓ Cuento 4 indexado: created
✓ Cuento 5 indexado: created
✓ Cuento 6 indexado: created
✓ Cuento 7 indexado: created
✓ Cuento 8 indexado: created


In [14]:
# Refrescar el index para que los documentos sean buscables
es.indices.refresh(index=INDEX_NAME)
print(f"\n {len(cuentos)} cuentos cargados exitosamente")


✓ 8 cuentos cargados exitosamente


#PARTE 4: CONSULTAS BÁSICAS

## 4.1_consultar todos los documentos

In [16]:
print("\n" + "="*60)
print("EJERCICIO 1: Búsqueda de todos los documentos")
print("="*60)

query1 = {
    "query": {
        "match_all": {}
    }
}
result = es.search(index=INDEX_NAME, body=query1)
print(f"Total de documentos: {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- {hit['_source']['autor']}: {hit['_source']['tipo_cuento']}")


EJERCICIO 1: Búsqueda de todos los documentos
Total de documentos: 8
- María González: infantil
- Carlos Ruiz: terror
- Ana Martínez: fantástico
- Roberto Silva: político
- Laura Ramírez: infantil
- Carlos Ruiz: terror
- Ana Martínez: fantástico
- Roberto Silva: político


## 4.2_Búsqueda por una columna (caracteristica) del documento

In [17]:
print("\n" + "="*60)
print("EJERCICIO 2: Búsqueda por tipo de cuento específico")
print("="*60)

query2 = {
    "query": {
        "term": {
            "tipo_cuento": "terror"
        }
    }
}

result = es.search(index=INDEX_NAME, body=query2)
print(f"Cuentos de terror encontrados: {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- '{hit['_source']['texto'][:80]}...'")
    print(f"  Autor: {hit['_source']['autor']}\n")


EJERCICIO 2: Búsqueda por tipo de cuento específico
Cuentos de terror encontrados: 2
- 'La casa en la colina estaba abandonada desde hace décadas. Los vecinos decían qu...'
  Autor: Carlos Ruiz

- 'El espejo antiguo apareció en el mercado de pulgas. Quien se miraba en él no veí...'
  Autor: Carlos Ruiz



In [19]:
print("EJERCICIO 2.1: Búsqueda por autor")
print("="*60)

query4 = {
    "query": {
        "term": {
            "autor": "Carlos Ruiz"
        }
    }
}

result = es.search(index=INDEX_NAME, body=query4)
print(f"Cuentos de Carlos Ruiz: {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- {hit['_source']['tipo_cuento'].upper()}: {hit['_source']['texto'][:80]}...")

EJERCICIO 2.1: Búsqueda por autor
Cuentos de Carlos Ruiz: 2
- TERROR: La casa en la colina estaba abandonada desde hace décadas. Los vecinos decían qu...
- TERROR: El espejo antiguo apareció en el mercado de pulgas. Quien se miraba en él no veí...


## 4.3_Búsqueda de texto con match

In [18]:
print("\n" + "="*60)
print("EJERCICIO 3: Búsqueda de texto con match")
print("="*60)

query3 = {
    "query": {
        "match": {
            "texto": "dragón bosque mágico"
        }
    }
}

result = es.search(index=INDEX_NAME, body=query3)
print(f"Documentos que contienen 'dragón', 'bosque' o 'mágico': {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- Score: {hit['_score']:.2f} | Tipo: {hit['_source']['tipo_cuento']}")
    print(f"  Texto: {hit['_source']['texto'][:100]}...\n")


EJERCICIO 3: Búsqueda de texto con match
Documentos que contienen 'dragón', 'bosque' o 'mágico': 3
- Score: 2.98 | Tipo: infantil
  Texto: Había una vez un pequeño conejo llamado Tambor que vivía en el bosque encantado. Un día decidió expl...

- Score: 1.82 | Tipo: fantástico
  Texto: En el reino de Eldoria, la princesa Luna poseía el poder de controlar las estrellas. Cuando el dragó...

- Score: 1.36 | Tipo: infantil
  Texto: Pepito el elefante no podía volar como los pájaros, pero tenía el corazón más grande del bosque. Cua...



## 4.4_Búsqueda con rango de fechas

In [20]:
print("\n" + "="*60)
print("EJERCICIO 4: Búsqueda con rango de fechas")
print("="*60)

query5 = {
    "query": {
        "range": {
            "fecha_creacion": {
                "gte": "2024-04-01",
                "lte": "2024-07-31"
            }
        }
    }
}

result = es.search(index=INDEX_NAME, body=query5)
print(f"Cuentos entre abril y julio 2024: {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- {hit['_source']['fecha_creacion']} | {hit['_source']['tipo_cuento']} | {hit['_source']['autor']}")



EJERCICIO 4: Búsqueda con rango de fechas
Cuentos entre abril y julio 2024: 4
- 2024-04-05 | político | Roberto Silva
- 2024-05-12 | infantil | Laura Ramírez
- 2024-06-18 | terror | Carlos Ruiz
- 2024-07-22 | fantástico | Ana Martínez


## 4.5_Búsqueda booleana compuesta

In [21]:
print("\n" + "="*60)
print("EJERCICIO 6: Búsqueda booleana compuesta")
print("="*60)

query6 = {
    "query": {
        "bool": {
            "must": [
                {"match": {"texto": "reino"}}
            ],
            "filter": [
                {"term": {"tipo_cuento": "fantástico"}}
            ]
        }
    }
}

result = es.search(index=INDEX_NAME, body=query6)
print(f"Cuentos fantásticos que mencionan 'reino': {result['hits']['total']['value']}")
for hit in result['hits']['hits']:
    print(f"- {hit['_source']['autor']}: {hit['_source']['texto'][:100]}...")


EJERCICIO 6: Búsqueda booleana compuesta
Cuentos fantásticos que mencionan 'reino': 1
- Ana Martínez: En el reino de Eldoria, la princesa Luna poseía el poder de controlar las estrellas. Cuando el dragó...


# PARTE 5: CONSULTAS CON AGREGACIONES

## 5.1_Contar cuentos por una columna

In [22]:
print("\n" + "="*60)
print("AGREGACIÓN 1: Contar cuentos por tipo")
print("="*60)

agg1 = {
    "size": 0,
    "aggs": {
        "cuentos_por_tipo": {
            "terms": {
                "field": "tipo_cuento",
                "size": 10
            }
        }
    }
}

result = es.search(index=INDEX_NAME, body=agg1)
for bucket in result['aggregations']['cuentos_por_tipo']['buckets']:
    print(f"- {bucket['key']}: {bucket['doc_count']} cuentos")


AGREGACIÓN 1: Contar cuentos por tipo
- fantástico: 2 cuentos
- infantil: 2 cuentos
- político: 2 cuentos
- terror: 2 cuentos


In [23]:
print("\n" + "="*60)
print("AGREGACIÓN 2: Contar cuentos por autor")
print("="*60)

agg2 = {
    "size": 0,
    "aggs": {
        "cuentos_por_autor": {
            "terms": {
                "field": "autor",
                "size": 10
            }
        }
    }
}

result = es.search(index=INDEX_NAME, body=agg2)
for bucket in result['aggregations']['cuentos_por_autor']['buckets']:
    print(f"- {bucket['key']}: {bucket['doc_count']} cuentos")


AGREGACIÓN 2: Contar cuentos por autor
- Ana Martínez: 2 cuentos
- Carlos Ruiz: 2 cuentos
- Roberto Silva: 2 cuentos
- Laura Ramírez: 1 cuentos
- María González: 1 cuentos


## 5.2_Histograma de fechas (por mes)

In [24]:
print("\n" + "="*60)
print("AGREGACIÓN 3: Histograma de fechas (por mes)")
print("="*60)

agg3 = {
    "size": 0,
    "aggs": {
        "cuentos_por_mes": {
            "date_histogram": {
                "field": "fecha_creacion",
                "calendar_interval": "month"
            }
        }
    }
}

result = es.search(index=INDEX_NAME, body=agg3)
for bucket in result['aggregations']['cuentos_por_mes']['buckets']:
    fecha = bucket['key_as_string']
    print(f"- {fecha}: {bucket['doc_count']} cuentos")


AGREGACIÓN 3: Histograma de fechas (por mes)
- 2024-01-01T00:00:00.000Z: 1 cuentos
- 2024-02-01T00:00:00.000Z: 1 cuentos
- 2024-03-01T00:00:00.000Z: 1 cuentos
- 2024-04-01T00:00:00.000Z: 1 cuentos
- 2024-05-01T00:00:00.000Z: 1 cuentos
- 2024-06-01T00:00:00.000Z: 1 cuentos
- 2024-07-01T00:00:00.000Z: 1 cuentos
- 2024-08-01T00:00:00.000Z: 1 cuentos


## 5.3_Estadísticas de longitud de texto

In [25]:
print("\n" + "="*60)
print("AGREGACIÓN 4: Estadísticas de longitud de texto")
print("="*60)

agg4 = {
    "size": 0,
    "aggs": {
        "estadisticas_texto": {
            "stats": {
                "script": {
                    "source": "doc['texto.keyword'].value.length()"
                }
            }
        }
    }
}
# Nota: Esta agregación requiere habilitar scripting
# Para simplificar, mostraremos otra agregación

agg4_alternativa = {
    "size": 0,
    "aggs": {
        "tipos_por_autor": {
            "terms": {
                "field": "autor"
            },
            "aggs": {
                "tipos": {
                    "terms": {
                        "field": "tipo_cuento"
                    }
                }
            }
        }
    }
}

print("Tipos de cuentos por autor (agregación anidada):")
result = es.search(index=INDEX_NAME, body=agg4_alternativa)
for bucket in result['aggregations']['tipos_por_autor']['buckets']:
    print(f"\n{bucket['key']} ({bucket['doc_count']} cuentos):")
    for tipo in bucket['tipos']['buckets']:
        print(f"  - {tipo['key']}: {tipo['doc_count']}")


AGREGACIÓN 4: Estadísticas de longitud de texto
Tipos de cuentos por autor (agregación anidada):

Ana Martínez (2 cuentos):
  - fantástico: 2

Carlos Ruiz (2 cuentos):
  - terror: 2

Roberto Silva (2 cuentos):
  - político: 2

Laura Ramírez (1 cuentos):
  - infantil: 1

María González (1 cuentos):
  - infantil: 1
